### Hautus-Rosenbrock Controllability Test

The Hautus-Rosenbrock test is a criterion for determining the controllability of a linear time-invariant (LTI) system given by the state-space representation:

$\dot{x} = Ax + Bu$

A system is **controllable** if and only if for every eigenvalue $\lambda_i$ of the matrix $A$, the rank of the concatenated matrix $[\lambda_i I - A \mid B]$ is equal to the number of states, $n$.

That is, for each $\lambda_i \in \text{eigenvalues}(A)$:

$\text{rank}([\lambda_i I - A \mid B]) = n$

Where:
- $A$ is the state matrix ($n \times n$)
- $B$ is the input matrix ($n \times m$)
- $I$ is the identity matrix ($n \times n$)
- $\lambda_i$ is an eigenvalue of $A$
- $n$ is the dimension of the state vector (number of states)

###Important

Run these three scripts before continuing:

In [1]:
import numpy as np
from scipy.linalg import eigvals
from numpy.linalg import matrix_rank
import sympy
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from scipy.linalg import eigvals
from scipy.integrate import odeint  # Added for odeint

print("Libraries imported successfully.")

Libraries imported successfully.


In [2]:
def format_complex_for_display(c_num):
    """Formats a complex number for display, omitting +0.j if imaginary part is zero."""
    if np.isclose(c_num.imag, 0): # Check if imaginary part is effectively zero
        return f"{c_num.real:.3f}"
    else:
        # Format real part
        real_str = f"{c_num.real:.3f}"
        # Format imaginary part with sign
        imag_str = f"{c_num.imag:.3f}j"

        if c_num.real == 0: # Pure imaginary
            return imag_str
        elif c_num.imag > 0:
            return f"{real_str}+{imag_str}"
        else: # c_num.imag < 0
            return f"{real_str}{imag_str}"


In [3]:
def calculate_symbolic_eigenvalues(A_np):
    print("\n### Symbolic Eigenvalue Calculation ###")

    # 1. Convert NumPy matrix A to SymPy matrix
    A_sym = sympy.Matrix(A_np)

    # 2. Define symbolic variable for lambda
    lambda_sym = sympy.symbols('lambda')

    # 3. Create identity matrix of the same size as A_sym
    n = A_sym.shape[0]
    I_sym = sympy.eye(n)

    # 4. Construct the symbolic matrix (lambda*I - A)
    symbolic_matrix = lambda_sym * I_sym - A_sym
    print("\nSymbolic Matrix (λI - A):")
    # Print using sympy's pretty printing for matrices
    sympy.pprint(symbolic_matrix)

    # 5. Calculate the characteristic polynomial (determinant of (lambda*I - A))
    characteristic_polynomial = symbolic_matrix.det()
    print("\nCharacteristic Polynomial: p(λ) = det(λI - A) = 0")
    sympy.pprint(characteristic_polynomial)

    # 6. Solve the characteristic polynomial for lambda_sym to find the eigenvalues
    symbolic_eigenvalues = sympy.solve(characteristic_polynomial, lambda_sym)
    print("\nSymbolic Eigenvalues (roots of the characteristic polynomial):")
    sympy.pprint(symbolic_eigenvalues)

    return symbolic_eigenvalues

print("Function 'calculate_symbolic_eigenvalues' defined.")

Function 'calculate_symbolic_eigenvalues' defined.


### Example System Matrices A and B

Below, are three example setups of a state matrix `A` and input matrix `B`. Copy whichever code block you like, and then move on. If you have your own `A` and `B` matrix, move past these examples to the next block. The system size can be up to 4x4.

**Example 1: Controllable 2x2 system**
```python
A = np.array([
    [0, 1],
    [-2, -3]
])
B = np.array([
    [0],
    [1]
])
```

**Example 2: Uncontrollable 2x2 system**
```python
A = np.array([
    [0, 0],
    [0, 0]
])
B = np.array([
    [1],
    [0]
])
```

**Example 3: Controllable 4x4 system**
```python
A = np.array([
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [-24, -50, -35, -10]
])
B = np.array([
    [0],
    [0],
    [0],
    [1]
])
```

### Defining your System Matrices

If you are using an example setup, paste it here, or define your own system matrices (run the block once you have made them). Again, the system size can be up to 4x4.

In [4]:
# Your A and B matrices
A = np.array([
    [1, 1, 0],
    [0, 1, 1],
    [0, 0, 2]
])
B = np.array([
    [0],
    [1],
    [0]
])

# Continue to the test

Once you have ran the code block with your desired matrices, continue below to see your matrices, their eigenvalues, and controllability.

In [5]:
print("Matrix A:")
print(A)
print("\nMatrix B:")
print(B)

n = A.shape[0] # Number of states
m = B.shape[1] # Number of inputs

if A.shape[0] != A.shape[1]:
    raise ValueError("Matrix A must be square.")
if B.shape[0] != n:
    raise ValueError("Number of rows in B must match number of states in A.")

print(f"\nSystem size (n): {n}")

Matrix A:
[[1 1 0]
 [0 1 1]
 [0 0 2]]

Matrix B:
[[0]
 [1]
 [0]]

System size (n): 3


### Calculate Eigenvalues of A

Once you have initialized everything, run the below code to see how to solve for the eigenvalues of A.

In [6]:
symbolic_eigenvalues_A_calculated = calculate_symbolic_eigenvalues(A)

eigenvalues_A = eigvals(A)

print("Eigenvalues of A:")
for i, val in enumerate(eigenvalues_A):
    print(f"  λ_{i+1} = {format_complex_for_display(val)}")


### Symbolic Eigenvalue Calculation ###

Symbolic Matrix (λI - A):
⎡λ - 1   -1      0  ⎤
⎢                   ⎥
⎢  0    λ - 1   -1  ⎥
⎢                   ⎥
⎣  0      0    λ - 2⎦

Characteristic Polynomial: p(λ) = det(λI - A) = 0
 3      2          
λ  - 4⋅λ  + 5⋅λ - 2

Symbolic Eigenvalues (roots of the characteristic polynomial):
[1, 2]
Eigenvalues of A:
  λ_1 = 1.000
  λ_2 = 1.000
  λ_3 = 2.000


### Perform Hautus-Rosenbrock Test

Now that you have your eigenvalues, all that's left is to perform the test! Run the below code to do so, and note the output for each eigenvalue.

In [8]:
def hautus_rosenbrock_test(A, B):
    n = A.shape[0]
    eigenvalues = eigvals(A)

    controllable_all_eigenvalues = True
    results = []

    print(f"Checking controllability for system with n={n} states.\n")

    for i, lambda_val in enumerate(eigenvalues):
        print(f"--- For eigenvalue λ_{i+1} = {format_complex_for_display(lambda_val)} ---")
        # Construct (lambda*I - A)
        lambda_I_minus_A = lambda_val * np.eye(n) - A

        # Construct the Hautus-Rosenbrock matrix [lambda*I - A | B]
        HR_matrix = np.hstack((lambda_I_minus_A, B))

        print("  [λI - A | B] matrix:")
        # Define a custom formatter for complex numbers for numpy array printing
        complex_formatter = {'complexfloat': lambda x: format_complex_for_display(x),
                             'float_kind': lambda x: format_complex_for_display(x + 0j)} # Ensure real floats are also formatted
        # print the matrix with custom formatting
        print(np.array2string(HR_matrix, formatter=complex_formatter, precision=4, suppress_small=True, separator='  '))

        # Calculate the rank
        rank_HR = matrix_rank(HR_matrix, tol=1e-9) # tolerance for numerical stability

        is_controllable_for_lambda = (rank_HR == n)
        controllable_all_eigenvalues = controllable_all_eigenvalues and is_controllable_for_lambda

        result_str = f"  Rank of [λI - A | B] = {rank_HR} (expected {n}). "
        if is_controllable_for_lambda:
            result_str += "Controllable for this eigenvalue."
        else:
            result_str += "NOT controllable for this eigenvalue."
        results.append(result_str)
        print(result_str)
        print("\n")

    print("--- Overall Controllability --- ")
    if controllable_all_eigenvalues:
        print("The system IS controllable (Hautus-Rosenbrock test passed for all eigenvalues).")
    else:
        print("The system IS NOT controllable (Hautus-Rosenbrock test failed for at least one eigenvalue).")

    return controllable_all_eigenvalues

# Run the test
is_system_controllable = hautus_rosenbrock_test(A, B)

Checking controllability for system with n=3 states.

--- For eigenvalue λ_1 = 1.000 ---
  [λI - A | B] matrix:
[[0.000  -1.000  0.000  0.000]
 [0.000  0.000  -1.000  1.000]
 [0.000  0.000  -1.000  0.000]]
  Rank of [λI - A | B] = 3 (expected 3). Controllable for this eigenvalue.


--- For eigenvalue λ_2 = 1.000 ---
  [λI - A | B] matrix:
[[0.000  -1.000  0.000  0.000]
 [0.000  0.000  -1.000  1.000]
 [0.000  0.000  -1.000  0.000]]
  Rank of [λI - A | B] = 3 (expected 3). Controllable for this eigenvalue.


--- For eigenvalue λ_3 = 2.000 ---
  [λI - A | B] matrix:
[[1.000  -1.000  0.000  0.000]
 [0.000  1.000  -1.000  1.000]
 [0.000  0.000  0.000  0.000]]
  Rank of [λI - A | B] = 2 (expected 3). NOT controllable for this eigenvalue.


--- Overall Controllability --- 
The system IS NOT controllable (Hautus-Rosenbrock test failed for at least one eigenvalue).


# Next Steps: Gaining an intuition of controllability through visualization

## User Input for 2x2 System Matrices

#### Instructions
1. Define a 2x2 matrix for the state matrix `A` and a 2x1 matrix for the input matrix `B`.
2. Run the code cell to set these matrices for further computations.

If you used a 2x2 system before, feel free to try it here to see more details. It is a little difficult to visualize a 4x4, and I did not have the time nor skill to code a 3x3 visualization.

In [9]:
# Define your 2x2 state matrix A
A = np.array([
    [0, 1],
    [-2, -3]
])
B = np.array([
    [0],
    [1]
])

print("Custom 2x2 system matrices A and B defined. Please ensure A is 2x2 and B is 2x1.")
print("A:\n", A)
print("B:\n", B)

Custom 2x2 system matrices A and B defined. Please ensure A is 2x2 and B is 2x1.
A:
 [[ 0  1]
 [-2 -3]]
B:
 [[0]
 [1]]


### Run Simulation Setup:

Before running the interactive visualization below, please ensure you have executed the simulation setup code block:

In [10]:
def open_loop_dynamics(x, t, A):
    """Open-loop system dynamics: dx/dt = Ax"""
    return A @ x

def closed_loop_dynamics(x, t, A, B, K):
    """Closed-loop system dynamics: dx/dt = (A - BK)x"""
    A_cl = A - B @ K
    return A_cl @ x

# Simulation parameters
t_sim = np.linspace(0, 10, 1000) # Time vector from 0 to 10 seconds, 1000 points
x0 = np.array([1, 1])           # Initial state (excites both modes)

# Simulate open-loop response
print("Simulating open-loop response...")
x_open_loop = odeint(open_loop_dynamics, x0, t_sim, args=(A,))
print("Open-loop simulation complete.")

# Simulate closed-loop response
print("\nSimulating closed-loop response...")
K_initial = np.array([[1.0, 0.0]]) # Use default K for initial closed-loop simulation
x_closed_loop = odeint(closed_loop_dynamics, x0, t_sim, args=(A, B, K_initial))
print("Closed-loop simulation complete.")

print("Simulation data for open-loop (x_open_loop) and closed-loop (x_closed_loop) systems generated.")

Simulating open-loop response...
Open-loop simulation complete.

Simulating closed-loop response...
Closed-loop simulation complete.
Simulation data for open-loop (x_open_loop) and closed-loop (x_closed_loop) systems generated.


## Interactive Visualization

This will provide a more comprehensive interactive visualization of the controller's effect on the system's modes. Remember, the core of controllability is that you can take the system to the origin from anywhere. In the 2x2 case, you are trying to make sure that both x1 and x2 go to zero. If that doesn't happen, the system is not controllable! Check controllability with the first part of this notebook if you want to know beforehand if the system is controllable.

In [11]:
def update_plot(k1_value, k2_value):
    # Construct the K matrix with the new k1_value and k2_value
    K_current = np.array([[k1_value, k2_value]])

    # Recalculate A_cl = A - BK
    A_cl_current = A - B @ K_current

    # Calculate closed-loop eigenvalues
    closed_loop_eigenvalues = eigvals(A_cl_current)

    # Calculate open-loop eigenvalues (A)
    open_loop_eigenvalues = eigvals(A)

    # Format eigenvalues for display
    formatted_cl_eigvals = [format_complex_for_display(val) for val in closed_loop_eigenvalues]
    formatted_ol_eigvals = [format_complex_for_display(val) for val in open_loop_eigenvalues]

    # 3b. Recalculate x_closed_loop with the new K matrix
    x_closed_loop_current = odeint(closed_loop_dynamics, x0, t_sim, args=(A, B, K_current))

    # 3c. Clear the current plot to prepare for the new one
    plt.close('all') # Close previous figures to avoid memory issues and display overlap

    # 3d. Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # 3e. On the first subplot, plot the open-loop and closed-loop responses for state x1
    ax1.plot(t_sim, x_open_loop[:, 0], label='Open-loop $x_1$')
    ax1.plot(t_sim, x_closed_loop_current[:, 0], label=f'Closed-loop $x_1$ (k1={k1_value:.1f}, k2={k2_value:.1f})', linestyle='--')
    ax1.set_xlabel('Time')
    ax1.set_ylabel('$x_1$')
    ax1.set_title('State $x_1$ over Time')
    ax1.legend()
    ax1.grid(True)

    # 3f. On the second subplot, plot the open-loop and closed-loop responses for state x2
    ax2.plot(t_sim, x_open_loop[:, 1], label='Open-loop $x_2$')
    ax2.plot(t_sim, x_closed_loop_current[:, 1], label=f'Closed-loop $x_2$ (k1={k1_value:.1f}, k2={k2_value:.1f})', linestyle='--')
    ax2.set_xlabel('Time')
    ax2.set_ylabel('$x_2$')
    ax2.set_title('State $x_2$ over Time')
    ax2.legend()
    ax2.grid(True)
    # Set fixed y-limits for x2 to emphasize its potentially constant nature
    ax2.set_ylim(min(np.min(x_open_loop[:, 1]), np.min(x_closed_loop_current[:, 1])) * 0.95, max(np.max(x_open_loop[:, 1]), np.max(x_closed_loop_current[:, 1])) * 1.05)

    # Display eigenvalues as text above the plots
    fig.suptitle(f"Open-loop Eigenvalues (A): {', '.join(formatted_ol_eigvals)}\nClosed-loop Eigenvalues (A-BK): {', '.join(formatted_cl_eigvals)}", fontsize=12)

    # Add titles, labels, legends, grids, and display the plot
    plt.tight_layout(rect=[0, 0.03, 1, 0.9]) # Adjust layout to make space for suptitle
    plt.show()

# 4. create the interactive plot
k1_slider = widgets.FloatSlider(
    min=-5.0,
    max=5.0,
    step=0.1,
    value=1.0, # Default value for k1
    description='k1 gain:'
)
k2_slider = widgets.FloatSlider(
    min=-5.0,
    max=5.0,
    step=0.1,
    value=0.0, # Default value for k2
    description='k2 gain:'
)

print("Adjust the sliders below to see the effect of different gain values k1 and k2 on the system's states and closed-loop eigenvalues:")
interactive_plot = widgets.interactive(update_plot, k1_value=k1_slider, k2_value=k2_slider)
display(interactive_plot)

Adjust the sliders below to see the effect of different gain values k1 and k2 on the system's states and closed-loop eigenvalues:


interactive(children=(FloatSlider(value=1.0, description='k1 gain:', max=5.0, min=-5.0), FloatSlider(value=0.0…

## Summary: Intuition from the Demonstration

This provides a clear image regarding uncontrollable modes in dynamic systems:

1.  **Uncontrollable Modes Remain Fixed**: The most important takeaway is that an uncontrollable mode, identified by a rank deficiency in the Hautus-Rosenbrock test for a specific eigenvalue, will always maintain its inherent dynamic behavior regardless of any state feedback control applied. In the 2x2 case, if one of the open-loop eigenvalues was 0, the closed-loop analysis will show that one eigenvalue remained fixed at 0, irrespective of the `k1` or `k2` gains.

2.  **Visual Confirmation**: The interactive plot shows that. While adjusting the `k1` gain dramatically changed the behavior of a controllable state – allowing it to decay rapidly or diverge based on the gain value – the uncontrollable state remained completely unaffected, maintaining its initial value. This shows that the controller has no handle over that specific part of the system's dynamics.

3.  **Limitations of Control**: This implies that if a system has an unstable uncontrollable mode (e.g., an eigenvalue with a positive real part), that instability cannot be corrected by state feedback, and the system will "blow up". Similarly, if an uncontrollable mode exhibits undesirable but stable behavior (like a persistent oscillation or a constant offset), it cannot be helped by the controller.

## If you are curious: General State Feedback Controller Derivation for 2x2 Systems

---

### 1. State Feedback Controller and Closed-Loop Dynamics

The general form of a linear state feedback controller is given by:

$u = -Kx$

Where $u$ is the control input, $K$ is the state feedback gain matrix, and $x$ is the state vector. Substituting this into the state-space equation $\dot{x} = Ax + Bu$ gives the closed-loop system dynamics:

$\dot{x} = Ax + B(-Kx) = (A - BK)x$

Here, $A_{cl} = (A - BK)$ is the closed-loop system matrix.

### 2. Defining General 2x2 Matrices

Let's consider general 2x2 system matrices $A$, a 2x1 input matrix $B$, and a 1x2 feedback gain matrix $K$:

$A = \begin{bmatrix} a_{11} & a_{12} \\ a_{21} & a_{22} \end{bmatrix}$

$B = \begin{bmatrix} b_1 \\ b_2 \end{bmatrix}$

$K = \begin{bmatrix} k_1 & k_2 \end{bmatrix}$

### 3. Deriving the Product $BK$

The product $BK$ is calculated by matrix multiplication:

$BK = \begin{bmatrix} b_1 \\ b_2 \end{bmatrix} \begin{bmatrix} k_1 & k_2 \end{bmatrix} = \begin{bmatrix} b_1 k_1 & b_1 k_2 \\ b_2 k_1 & b_2 k_2 \end{bmatrix}$

### 4. Deriving the Closed-Loop System Matrix $A_{cl} = A - BK$

Now, we subtract $BK$ from $A$ to get the closed-loop system matrix $A_{cl}$:

$A_{cl} = A - BK = \begin{bmatrix} a_{11} & a_{12} \\ a_{21} & a_{22} \end{bmatrix} - \begin{bmatrix} b_1 k_1 & b_1 k_2 \\ b_2 k_1 & b_2 k_2 \end{bmatrix}$

$A_{cl} = \begin{bmatrix} a_{11} - b_1 k_1 & a_{12} - b_1 k_2 \\ a_{21} - b_2 k_1 & a_{22} - b_2 k_2 \end{bmatrix}$

### 5. Computing the Characteristic Equation $det(sI - A_{cl}) = 0$

The eigenvalues of the closed-loop system are the roots of its characteristic equation, $det(sI - A_{cl}) = 0$. First, we form the matrix $(sI - A_{cl})$:

$sI - A_{cl} = s \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix} - \begin{bmatrix} a_{11} - b_1 k_1 & a_{12} - b_1 k_2 \\ a_{21} - b_2 k_1 & a_{22} - b_2 k_2 \end{bmatrix}$

$sI - A_{cl} = \begin{bmatrix} s - (a_{11} - b_1 k_1) & -(a_{12} - b_1 k_2) \\ -(a_{21} - b_2 k_1) & s - (a_{22} - b_2 k_2) \end{bmatrix}$

Now, we compute the determinant:

$det(sI - A_{cl}) = (s - (a_{11} - b_1 k_1))(s - (a_{22} - b_2 k_2)) - (-(a_{12} - b_1 k_2))(-(a_{21} - b_2 k_1)) = 0$

Expanding this, we get a quadratic equation in $s$:

$s^2 - s(a_{11} - b_1 k_1 + a_{22} - b_2 k_2) + (a_{11} - b_1 k_1)(a_{22} - b_2 k_2) - (a_{12} - b_1 k_2)(a_{21} - b_2 k_1) = 0$

This can be written in the standard form $s^2 + \alpha_1 s + \alpha_0 = 0$, where:

$\alpha_1 = -(a_{11} + a_{22}) + (b_1 k_1 + b_2 k_2)$

$\alpha_0 = (a_{11} - b_1 k_1)(a_{22} - b_2 k_2) - (a_{12} - b_1 k_2)(a_{21} - b_2 k_1)$

### 6. Analysis of How $k_1$ and $k_2$ Affect Closed-Loop Eigenvalues

The closed-loop eigenvalues are the roots of the characteristic polynomial $s^2 + \alpha_1 s + \alpha_0 = 0$. From the expressions for $\alpha_1$ and $\alpha_0$ above, we can observe:

*   **Dependence on $k_1$ and $k_2$**: Both coefficients $\alpha_1$ and $\alpha_0$ are functions of $k_1$ and $k_2$. This means that by strategically choosing values for $k_1$ and $k_2$, we can manipulate these coefficients, and consequently, the roots (eigenvalues) of the characteristic equation.
*   **Controllability**: If the system is controllable, it implies that we can choose $k_1$ and $k_2$ to arbitrarily place the closed-loop eigenvalues to any desired locations in the complex plane. This is often referred to as pole placement.
*   **Uncontrollable Modes**: If the system is *uncontrollable*, it means that there are certain combinations of the system parameters ($A$ and $B$) for which the coefficients $\alpha_1$ and $\alpha_0$ cannot be arbitrarily changed by $k_1$ and $k_2$. Specifically, if an eigenvalue corresponds to an uncontrollable mode, that eigenvalue will remain fixed at its open-loop value, regardless of the choice of $k_1$ and $k_2$. In a 2x2 system with one uncontrollable mode, one of the closed-loop eigenvalues will be fixed, while the other can be shifted by $k_1$ and $k_2$.